In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import re

from tqdm import tqdm
tqdm.pandas()  # This adds the tqdm progress bar to pandas operations
from openai import OpenAI
client = OpenAI()
from dotenv import load_dotenv
import json
ret = load_dotenv()

In [6]:
def page_source_loader(input_url):
    service = Service(executable_path=ChromeDriverManager().install())
    chrome_options = Options()
    ua = UserAgent()
    user_agent = ua.random
    chrome_options.add_argument(f'user-agent={user_agent}')
    # chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(input_url)
    staleness_counter = 0

    while staleness_counter <= 2:
        '''
        While loop to load all results for the search query by finding and clicking on the "Load More" button unless
        the button doesn't exist i.e., all results loaded, no more results to load. Without this, only the first 20 listings
        will be scraped
        '''
        try:
            load_more_button = (
                WebDriverWait(driver, 20)
                .until(EC.visibility_of_element_located((By.XPATH, '//button[contains(@data-aut-id, "btnLoadMore")]')))
            )
            load_more_button_.click()
        except (NoSuchElementException, TimeoutException):
            break
        except StaleElementReferenceException:
            staleness_counter += 1
            continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # print(soup.prettify
    sanitized_name = re.sub(r'[^\w\-_\.]', '_', input_url)
    file_name = f"htmls/{sanitized_name}.html"

    with open(file_name, "w", encoding="utf-8") as file:
        file.write(soup.prettify())

    
    # Extracting the title
    title = ""
    if soup.find('h1', class_='_2iMMO'):
        title = soup.find('h1', class_='_2iMMO').text
    elif soup.find('h1', class_='_1Zwvv'):
        title = soup.find('h1', class_='_1Zwvv').text
    
    # Extracting the price
    price = ""
    if soup.find('div', class_='_1uqlc'):
        price = soup.find('div', class_='_1uqlc').text
    elif soup.find('div', class_='_1eOrM'):
        price = soup.find('div', class_='_1eOrM').text
    
    # Extracting the description
    description = ""
    if soup.find('div', class_='_2CNvW'):
        description = soup.find('div', class_='_2CNvW').text
    elif soup.find('div', class_='_2BGfr'):
        description = soup.find('div', class_='_2BGfr').text
    
    # Extracting the seller using regex
    seller = ""
    if soup.find('div', class_='HtnZ6'):
        seller = soup.find('div', class_='HtnZ6').text
    elif soup.find('div', class_='_1aCTB'):
        seller = soup.find('div', class_='_1aCTB').text

    matches = re.findall(r'"addressLine1":"([^"]+)",\s*"addressLine2"', soup.prettify())

    # Filter out 'null' matches and only extract valid places
    valid_places = [match for match in matches if match != 'null']
    
    if valid_places:
        place = valid_places
    else:
        place=""
    
    matches = re.findall(r'"phone":"(.*?)",\s*"additional_services"', soup.prettify())

    if matches:
        phone_number = [match for match in matches]
    else:
        phone_number = ""

    date_pattern = r'<div class="_1Qxza">\s*([\d-]+-[A-Z]+-[\d]+)\s*</div>|<div class="_3VRXh">\s*([\d-]+-[A-Z]+-[\d]+)\s*</div>'
    date_match = re.search(date_pattern, soup.prettify())
    # print(match.group(1))
    if date_match:
        date =  date_match.group(1)
    else:
        date=""
    
    data = {
        "url":[input_url],
        "title": [title],
        "price": [price],
        "description": [description],
        "seller": [seller],
        "place": [place],
        "phone_number": [phone_number],
        "date":[date]
    }
    # print(data)
    return pd.DataFrame(data)




def date_extractor(input_url):
    service = Service(executable_path=ChromeDriverManager().install())
    chrome_options = Options()
    ua = UserAgent()
    user_agent = ua.random
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(input_url)
    staleness_counter = 0

    while staleness_counter <= 2:
        '''
        While loop to load all results for the search query by finding and clicking on the "Load More" button unless
        the button doesn't exist i.e., all results loaded, no more results to load. Without this, only the first 20 listings
        will be scraped
        '''
        try:
            load_more_button = (
                WebDriverWait(driver, 20)
                .until(EC.visibility_of_element_located((By.XPATH, '//button[contains(@data-aut-id, "btnLoadMore")]')))
            )
            load_more_button_.click()
        except (NoSuchElementException, TimeoutException):
            break
        except StaleElementReferenceException:
            staleness_counter += 1
            continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    date_pattern = r'<div class="_1Qxza">\s*([\d-]+-[A-Z]+-[\d]+)\s*</div>|<div class="_3VRXh">\s*([\d-]+-[A-Z]+-[\d]+)\s*</div>'
    match = re.search(date_pattern, soup.prettify())
    # print(match.group(1))
    if match:
        return match.group(1)

    return ""




In [3]:
df = pd.read_csv("kozhikode_g4058877.csv")

In [4]:
df['links'] = "https://www.olx.in/en-in/item"+df['car_link']

In [5]:
df.values

array([['/cars-c84-used-mercedes-benz-e-class-in-thondayad-kozhikode-iid-1779967365',
        'https://www.olx.in/en-in/item/cars-c84-used-mercedes-benz-e-class-in-thondayad-kozhikode-iid-1779967365'],
       ['/cars-c84-used-mercedes-benz-e-class-in-chalappuram-kozhikode-iid-1791596052',
        'https://www.olx.in/en-in/item/cars-c84-used-mercedes-benz-e-class-in-chalappuram-kozhikode-iid-1791596052'],
       ['/cars-c84-used-hyundai-grand-i10-in-palat-nagar-kozhikode-iid-1789760974',
        'https://www.olx.in/en-in/item/cars-c84-used-hyundai-grand-i10-in-palat-nagar-kozhikode-iid-1789760974'],
       ['/cars-c84-used-maruti-suzuki-swift-in-district-jail-kozhikode-kozhikode-iid-1791996428',
        'https://www.olx.in/en-in/item/cars-c84-used-maruti-suzuki-swift-in-district-jail-kozhikode-kozhikode-iid-1791996428'],
       ['/cars-c84-used-nissan-micra-active-in-malaparamba-kozhikode-iid-1780039732',
        'https://www.olx.in/en-in/item/cars-c84-used-nissan-micra-active-in-malapa

In [6]:
df.sample(2)

,car_link,links
411,/cars-c84-used-maruti-suzuki-swift-in-areekkad...,https://www.olx.in/en-in/item/cars-c84-used-ma...
49,/cars-c84-used-nissan-micra-in-district-jail-k...,https://www.olx.in/en-in/item/cars-c84-used-ni...


In [11]:
car_df = pd.read_csv("kozhikode_cars.csv")
for i, row in tqdm(df.iterrows(), desc="Loading Cars", total=len(df)):
    if row['links'] not in car_df.url.tolist():

        car_data = page_source_loader(row['links'])
        car_df = pd.concat([car_df, car_data], ignore_index=True)
        car_df.to_csv("kozhikode_cars.csv", index=False)


Loading Cars: 100%|█████████████████████████| 500/500 [3:29:05<00:00, 25.09s/it]
